In [9]:
using Laplacians

In [10]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/IPM_laplacian/Laplacians.jl/src/flowUtils.jl")

cutCapacity

In [11]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/IPM_laplacian/Laplacians.jl/src/min_cost_flow.jl")
#include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
#include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")

makeAdj (generic function with 1 method)

We introduce a data type for minimum cost flow problems.
It has an edge list, a vector of capacities of edges, a vector of costs, and a vector of demands.
We test this with the graph on 4 vertices with source 1 and destination 4.
We try to flow 1 unit of flow.  
The capacities on all edges are 0.7.
The costs on route 1-2-4 are 1, and on route 1-3-4 are 2.
So, it should flow most on route 1-2-4, as it does.

The following cell computes the costs and check that the demands are satisfied.

Now, let's try computing this using JuMP, an optimization package for Julia.

In [12]:
using JuMP
using Clp

In [13]:
function MCFjump(mcfp::MCFproblem)
    edge_list = mcfp.edge_list
    m = size(edge_list,1)
    n = maximum(edge_list)
    B = sparse(collect(1:m), edge_list[:,1], 1.0, m, n) -
      sparse(collect(1:m), edge_list[:,2], 1.0, m, n)
 

    mod = Model(solver=ClpSolver())
    @variable(mod, x[1:m] >= 0)
    @constraint(mod, x .<= mcfp.capacities)
    @constraint(mod, B'*x .== mcfp.demands)
    @objective(mod, Min, (mcfp.costs'*x)[1])
    @time status = solve(mod)

    f = getvalue(x)
    return f
    
end

MCFjump (generic function with 1 method)

In [14]:
f = MCFjump(mcfp)
reportMCFresults(mcfp,f)

LoadError: UndefVarError: mcfp not defined

We would like to compare the performance of our code to standard codes on some benchmark examples.  These benchmark examples usually come in the Dimacs format.  
In this notebook, we use the example "goto_8_08a.min"
from <a href="http://lime.cs.elte.hu/~kpeter/data/mcf/goto/">http://lime.cs.elte.hu/~kpeter/data/mcf/goto/</a>.

In [15]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_08a.min")

LoadError: SystemError: opening file /Users/kimonfountoulakis/Desktop/temp/goto_8_08a.min: No such file or directory

In [8]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/IPM_laplacian/Laplacians.jl/src/min_cost_flow.jl")
#include("/Users/spielman/Laplacians/src/primalDualIPM.jl")
#include("/Users/spielman/Laplacians/src/max_flow_IPM.jl")

#sol = min_cost_flow(mcfp, lapSolver = (h->augTreeLap(h, tol=1e-12, verbose=true, maxits=1000)), tol = 1e-10, reg_p = 0, reg_d = 0, tol_ref = 1.0e-1)
@time sol = min_cost_flow(mcfp, lapSolver = cholLap, tol = 1e-13, reg_p = 1.0e-10, reg_d = 1.0e-10, tol_ref = 1.0e-8)
flow = sol[1]
reportMCFresults(mcfp,flow)

LoadError: could not open file /Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/Laplacians.jl/src/min_cost_flow.jl

As a sanity check, we compare the results from MCFjump.

This is very similar.  But, JuMP computed a flow of a slightly lower cost.
This should not be surprising, because we haven't figured out exactly what tol we want.
It might help to know that all the numbers in the Dimacs problems are integers.

For comparison, we should also try the code from "Lemon", available at

<a href="http://lemon.cs.elte.hu/trac/lemon">http://lemon.cs.elte.hu/trac/lemon</a>

It was easy to install on my Mac (I just needed cmake, which I got by `brew install cmake`)

Here is a transcript of running it:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_08a.min 
Problem type: min
Num of nodes: 256
Num of arcs:  2048

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00467205s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 7.70092e-05s
Run NetworkSimplex: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00143886s

Feasible flow: found
Min flow cost: 560870539
```

## More Tests

Let's just do more tests on the GOTO problems, just to get a feel for the speeds involved.  Keep in mind that different problems will have very different run times, and that our code can be sped up a lot.

The Laplacian systems generated by the IPMs can have edge weights varying by 12 orders of magnitude!  This causes some of the solvers to run into numerical issues.  This is interesting to me, as it points out a place where our solvers need more development.

In [11]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_13a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 7362 7771; 7771 8180],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0,148449.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0,50.0],[148449.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [12]:
flow = MCFjump(mcfp)
reportMCFresults(mcfp, flow)

 42.347159 seconds (144.72 k allocations: 27.155 MB, 0.34% gc time)
Cost: 1.8217956686e10
Min flow: 0.0
Min slack: 0.0
Error on demands: 0.0


In [13]:
sol = min_cost_flow(mcfp, lapSolver = (h->cholSDDM(h, tol=1e-12, verbose=true, maxits=1000)), tol = 1e-13, reg_p = 1.0e-10, reg_d = 1.0e-10, tol_ref = 1.0e-8)
flow = sol[1]
reportMCFresults(mcfp,flow)

number of nodes =8192, number of edges=65536
maximum x.*s =3.112202e+10, minimum x.*s=4.808836e+04, mu=2.121781e+09
maximum (u-x).*z =1.511367e+09, minimum (u-x).*z=4.761271e+04, mu=2.121781e+09

Iteration 1, ||r_p||/||b||=2.158152e+00, ||r_d||/||c||=1.706092e+02, rel. gap=2.867166e-02, alpha=1.000000e+00
maximum theta =5.536111e+05, minimum theta=1.555805e+01, cond theta=3.558358e+04
maximum theta inverse =6.427540e-02, minimum theta inverse =1.806322e-06, cond theta inverse=3.558358e+04
normal eq. residual =1.717251e-09
pred. norm(res_p_saddle_1) =1.709826e-09, norm(res_d_saddle_1)=6.919843e-09
normal eq. residual =2.238521e-08
corr. norm(res_p_saddle_1) =2.362271e-08, norm(res_d_saddle_1)=5.343469e-06
normal eq. residual =1.491712e-16
corr. norm(res_p_saddle_2) =4.208029e-10, norm(res_d_saddle_2)=1.039170e-05
maximum x.*s =3.111449e+10, minimum x.*s=3.644113e+03, mu=2.121487e+09
maximum (u-x).*z =1.511226e+09, minimum (u-x).*z=4.672630e+05, mu=2.121487e+09

Iteration 2, ||r_p||/||b|

Here is what Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_13a.min 
Problem type: min
Num of nodes: 8192
Num of arcs:  65536

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.08s, s: 0s, cu: 0s, cs: 0s, real: 0.0842741s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00165105s
Run NetworkSimplex: u: 0.43s, s: 0s, cu: 0s, cs: 0s, real: 0.439616s

Feasible flow: found
Min flow cost: 18217956686
```

It was absurdly faster.  But, I won't worry about that yet.


Here is how Goldberg's CS2 code does, 
which I obtained from

<a href="https://github.com/iveney/cs2">https://github.com/iveney/cs2</a>

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_13a.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m2.425s
user	0m2.344s
sys	0m0.027s

Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:            8192     arcs:            65536
c scale-factor:       12     cut-off-factor:   79.1
c
c time:             2.25     cost:      18217956686
c refines:             4     discharges:    3409417
c pushes:        3621130     relabels:      1381573
```

## goto_8_14b.min

In [14]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_14b.min")

flow = MCFjump(mcfp)
reportMCFresults(mcfp, flow)

187.478247 seconds (114 allocations: 41.630 MB, 0.01% gc time)
Cost: 4.1834525746e10
Min flow: 0.0
Min slack: 0.0
Error on demands: 0.0


Some of these codes run into numerical issues at very low tolerances.  So, I will be more generous for this run.

In [59]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->cholSDDM(h, tol=1e-12, verbose=true, maxits=1000)), tol = 1e-12, reg_p = 1.0e-10, reg_d = 1.0e-10, tol_ref = 1.0e-8)
flow = sol[1]
reportMCFresults(mcfp, flow)

number of nodes =16384, number of edges=131072
maximum x.*s =5.830516e+10, minimum x.*s=6.598886e+04, mu=3.903648e+09
maximum (u-x).*z =2.352882e+09, minimum (u-x).*z=6.540233e+04, mu=3.903648e+09

Iteration 1, ||r_p||/||b||=2.485507e+00, ||r_d||/||c||=2.358563e+02, rel. gap=2.538213e-02, alpha=1.000000e+00
maximum theta =7.554044e+05, minimum theta=1.842139e+01, cond theta=4.100692e+04
maximum theta inverse =5.428473e-02, minimum theta inverse =1.323794e-06, cond theta inverse=4.100692e+04
normal eq. residual =2.544241e-09
pred. norm(res_p_saddle_1) =2.558001e-09, norm(res_d_saddle_1)=1.149337e-08
normal eq. residual =1.460648e-18
pred. norm(res_p_saddle_2) =3.654712e-10, norm(res_d_saddle_2)=2.935013e-08
normal eq. residual =2.243090e-07
corr. norm(res_p_saddle_1) =2.690509e-07, norm(res_d_saddle_1)=1.280708e-05
normal eq. residual =7.199541e-17
corr. norm(res_p_saddle_2) =9.240011e-10, norm(res_d_saddle_2)=2.032961e-05
maximum x.*s =5.829457e+10, minimum x.*s=5.003410e+03, mu=3.9032

If I try tol = 1e-4, that solver dies due to numerical issues.  

And, here is Lemon:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_14b.min 
Problem type: min
Num of nodes: 16384
Num of arcs:  131072

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.18s, s: 0s, cu: 0s, cs: 0s, real: 0.187166s
Setup NetworkSimplex class: u: 0s, s: 0s, cu: 0s, cs: 0s, real: 0.00391507s
Run NetworkSimplex: u: 1.6s, s: 0.01s, cu: 0s, cs: 0s, real: 1.61133s

Feasible flow: found
Min flow cost: 41834525746
```


And, here is CS2:

```
Dans17:cs2 spielman$ time ./cs2 < ~/tmp/goto_8_14b.min > out.txt
warning: this program uses gets(), which is unsafe.

real	0m22.478s
user	0m21.870s
sys	0m0.096s
Dans17:cs2 spielman$ head out.txt 
c CS 4.6
c Commercial use requires a licence
c contact igsys@eclipse.net
c
c nodes:           16384     arcs:           131072
c scale-factor:       12     cut-off-factor:  107.3
c
c time:            21.70     cost:      41834525746
c refines:             5     discharges:    9131788
c pushes:        9571379     relabels:      3496218
```

## goto_8_16a
Lemon (which is implementing a network simplex code) starts to scale badly when given this problem, as does CS2.
Our scales as we would expect.
This problem is 4 times bigger than `goto_8_14b`.

I stopped CS2 after 7 minutes, because it was driving me nuts.
Here is how Lemon did:

```
Dans17:tools spielman$ ./dimacs-solver ~/tmp/goto_8_16a.min 
Problem type: min
Num of nodes: 65536
Num of arcs:  524288

Sum of supply values: 0
GEQ supply contraints are used for NetworkSimplex

Read the file: u: 0.68s, s: 0.01s, cu: 0s, cs: 0s, real: 0.711415s
Setup NetworkSimplex class: u: 0.01s, s: 0.01s, cu: 0s, cs: 0s, real: 0.015645s
Run NetworkSimplex: u: 80.66s, s: 0.26s, cu: 0s, cs: 0s, real: 81.4035s
```

In [15]:
mcfp = readDimacsMCF("/Users/kimonfountoulakis/Desktop/temp/goto_8_16a.min")

MCFproblem{Float64,Int64}([1 2; 1 3; … ; 62244 63882; 63882 65520],[8642.0,741.0,453.0,62.0,341.0,1029.0,2640.0,1762.0,155.0,31.0  …  331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0,331827.0],[28.0,714.0,451.0,822.0,0.0,7.0,518.0,29.0,298.0,973.0  …  25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0,25.0],[331827.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0  …  0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0])

In [16]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->cholSDDM(h, tol=1e-12, verbose=true, maxits=1000)), tol = 1e-10, reg_p = 1.0e-10, reg_d = 1.0e-10, tol_ref = 1.0e-8)
flow = sol[1]
reportMCFresults(mcfp, flow)

number of nodes =65536, number of edges=524288
maximum x.*s =1.598429e+11, minimum x.*s=1.093886e+05, mu=1.043113e+10
maximum (u-x).*z =4.322689e+09, minimum (u-x).*z=1.087647e+05, mu=1.043113e+10

Iteration 1, ||r_p||/||b||=2.919599e+00, ||r_d||/||c||=3.948697e+02, rel. gap=1.574089e-02, alpha=1.000000e+00
maximum theta =1.245791e+06, minimum theta=2.662205e+01, cond theta=4.679544e+04
maximum theta inverse =3.756285e-02, minimum theta inverse =8.027031e-07, cond theta inverse=4.679544e+04
normal eq. residual =4.947619e-09
pred. norm(res_p_saddle_1) =5.199137e-09, norm(res_d_saddle_1)=2.888544e-08
normal eq. residual =1.317185e-18
pred. norm(res_p_saddle_2) =5.880804e-10, norm(res_d_saddle_2)=7.056396e-08
normal eq. residual =5.406400e-07
corr. norm(res_p_saddle_1) =4.782881e-07, norm(res_d_saddle_1)=6.403398e-05
normal eq. residual =9.618029e-16
corr. norm(res_p_saddle_2) =3.769622e-09, norm(res_d_saddle_2)=1.231647e-04
maximum x.*s =1.598248e+11, minimum x.*s=8.308912e+03, mu=1.0430

In [8]:
include("/Users/kimonfountoulakis/Desktop/Box Sync/University/Berkeley/postdoc/myPapers/Laplacians.jl/src/matlabSolvers.jl")

>> >> >> Connected to Matlab!


matlabCmgLap

In [9]:
@time sol = min_cost_flow(mcfp, lapSolver = (h->matlabCmgLap(h, tol=1e-8)), tol=1)
flow = sol[1]

Iteration 1, ||r_p||=1370095.345474, ||r_d||=116864630.148304, mu=20862256364.106258
>> >> >> 

Level 2. Graph vertices: 21992 
Density : 21.3993
Reduction factor in vertices : 2.97995
Reduction factor in edges    : 2.08458
*****************
Level 3. Graph vertices: 7408 
Density : 29.8799
Reduction factor in vertices : 2.96868
Reduction factor in edges    : 2.12611
*****************
Level 4. Graph vertices: 2408 
Density : 35.6944
Reduction factor in vertices : 3.07641
Reduction factor in edges    : 2.57527
*****************
Level 5. Graph vertices: 582 
Density : 33.8866
Reduction factor in vertices : 4.13746
Reduction factor in edges    : 4.35818
*****************
Level 6. Graph vertices: 198 
Density : 30.7475
Reduction factor in vertices : 2.93939
Reduction factor in edges    : 3.23949
*****************
Hierarchy constructed in 0.220000 seconds
>> >> >> 

Level 2. Graph vertices: 21992 
Density : 21.3993
Reduction factor in vertices : 2.97995
Reduction factor in edges    : 2.08458

524288-element Array{Float64,1}:
   8642.0      
    741.0      
    453.0      
     61.9999   
    340.993    
   1029.0      
   2640.0      
   1762.0      
    155.0      
     30.9999   
   9912.99     
   1070.0      
   9689.0      
      ⋮        
  75100.1      
  75095.1      
  83789.1      
  92948.1      
      1.01559e5
      1.06557e5
      1.0419e5 
 107967.0      
 120469.0      
 125774.0      
 133627.0      
 143580.0      

In [10]:
reportMCFresults(mcfp, flow)

Cost: 2.0566500432805237e11
Min flow: 1.1533763144431898e-7
Min slack: 1.1589416715196421e-7
Error on demands: 1.676119310230133
